# AlexNet in Keras

In this notebook, we leverage an [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks)-like deep, convolutional neural network to classify flowers into the 17 categories of the [Oxford Flowers](http://www.robots.ox.ac.uk/~vgg/data/flowers/17/) data set. Derived from [this earlier notebook](https://github.com/the-deep-learners/TensorFlow-LiveLessons/blob/master/notebooks/old/L3-3b__TFLearn_AlexNet.ipynb).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/the-deep-learners/deep-learning-illustrated/blob/master/notebooks/alexnet_in_keras.ipynb)

#### Load dependencies

In [1]:
%pip install -r /workspace/requirements_cpu.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 67.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=114921849c3df35bd8f38e09cee22b44ab7c2e8a0234cac35bd429613cab1d91
  Stored 

In [1]:
import tensorflow as tf
tf.__version__

2023-01-28 23:16:46.509885: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'2.11.0'

In [16]:
#import keras   for Jetson
#from keras.models import Sequential # For Jetson
from tensorflow.keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D # For Jetson
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
#from keras.layers.normalization import BatchNormalization # For Jetson
from tensorflow.keras.layers import BatchNormalization

#### Load *and preprocess* data

In [17]:
import tflearn.datasets.oxflower17 as oxflower17
X, Y = oxflower17.load_data(one_hot=True)

#### Design neural network architecture

In [18]:
model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(17, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 26, 26, 96)       384       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 22, 22, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 10, 256)      0         
 2D)                                                             
                                                      

#### Configure model

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 


#### Train!

In [24]:
model.fit(X, Y, batch_size=64, epochs=100, verbose=1, validation_split=0.1, shuffle=True)

Train on 1224 samples, validate on 136 samples
Epoch 1/100
1224/1224 [==============================] - 15s 12ms/sample - loss: 1.9563 - acc: 0.5000 - val_loss: 5.4127 - val_acc: 0.2353
Epoch 2/100
1224/1224 [==============================] - 15s 12ms/sample - loss: 1.6974 - acc: 0.5441 - val_loss: 4.6127 - val_acc: 0.2279
Epoch 3/100
1224/1224 [==============================] - 15s 12ms/sample - loss: 1.7619 - acc: 0.4975 - val_loss: 3.3280 - val_acc: 0.3456
Epoch 4/100
1224/1224 [==============================] - 15s 12ms/sample - loss: 1.6082 - acc: 0.5907 - val_loss: 3.6455 - val_acc: 0.3162
Epoch 5/100
1224/1224 [==============================] - 15s 12ms/sample - loss: 1.5566 - acc: 0.5980 - val_loss: 2.5124 - val_acc: 0.4706
Epoch 6/100
1224/1224 [==============================] - 15s 12ms/sample - loss: 1.7667 - acc: 0.5441 - val_loss: 4.7255 - val_acc: 0.3235
Epoch 7/100
1224/1224 [==============================] - 15s 13ms/sample - loss: 1.6807 - acc: 0.5686 - val_loss: 4.325